# Starbucks Capstone Challenge

## Problem statement

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

The task in this project is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. Informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

Transactional data show user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Also, someone using the app might make a purchase through the app without having received an offer or seen an offer.


## Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


In [699]:
import zipfile
import pandas as pd

import numpy as np
import math
import json
import datetime
from ast import literal_eval

import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# laod zip files
portfolio_zip = zipfile.ZipFile('data/portfolio.zip')
profile_zip = zipfile.ZipFile('data/profile.zip')
transcript_zip = zipfile.ZipFile('data/transcript.zip')

# read in the json files
portfolio_orig = pd.read_json(portfolio_zip.open('portfolio.json'), orient='records', lines=True)
profile_orig = pd.read_json(profile_zip.open('profile.json'), orient='records', lines=True)
transcript_orig = pd.read_json(transcript_zip.open('transcript.json'), orient='records', lines=True)

# Data Cleaning and Exploration

## Offer Portfolio

In [700]:
portfolio_orig.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [701]:
# Define mapping for offer_ids to more readable ids
offer_mapping = {
        'ae264e3637204a6fb9bb56bc8210ddfd' : 'B1',
        '4d5c57ea9a6940dd891ad53e9dbe8da0' : 'B2',
        '3f207df678b143eea3cee63160fa8bed' : 'I1',
        '9b98b8c7a33c4b65b9aebfe6a799e6d9' : 'B3',
        '0b1e1539f2cc45b7b9fa7c272da2e1d7' : 'D1',
        '2298d6c36e964ae4a3e7e9706d1fb8c2' : 'D2',
        'fafdcd668e3743c1bb461111dcafc2a4' : 'D3',
        '5a8bc65990b245e5a138643cd4eb9837' : 'I2',
        'f19421c1d4aa40978ebb69ca19b0e20d' : 'B4',
        '2906b810c7d4411798c6938adc9daaa5' : 'D4'
    }

In [702]:
def clean_portfolio (portfolio_df):
    '''
    Cleans the portfolio data by assigning clearer offer IDs, renaming 'id' to 'offer_id', 
    converting offer duration from days to hours, and expanding the 'channels' list to dedicated variables.
    
    Returns cleaned portfolio data.
    '''
    portfolio = portfolio_df.copy()
    
    # Rename 'id' to 'offer_id'
    portfolio.rename(columns={'id':'offer_id'}, inplace=True)
     
    # Convert 'duration' from days to hours
    portfolio['duration'] = portfolio['duration'] * 24
    
    # Assign more readable offer ids
    portfolio['offer_id'] = portfolio['offer_id'].map(offer_mapping)
    
    # Expand channels list
    portfolio['web'] = portfolio['channels'].astype(str).str.contains('web').astype(int)
    portfolio['email'] = portfolio['channels'].astype(str).str.contains('email').astype(int)
    portfolio['mobile'] = portfolio['channels'].astype(str).str.contains('mobile').astype(int)
    portfolio['social'] = portfolio['channels'].astype(str).str.contains('social').astype(int)
    portfolio.drop(columns='channels', inplace=True)
    
    return portfolio

portfolio = clean_portfolio(portfolio_orig)

In [703]:
portfolio.head()

,reward,difficulty,duration,offer_type,offer_id,web,email,mobile,social
0,10,10,168,bogo,B1,0,1,1,1
1,10,10,120,bogo,B2,1,1,1,1
2,0,0,96,informational,I1,1,1,1,0
3,5,5,168,bogo,B3,1,1,1,0
4,5,20,240,discount,D1,1,1,0,0


## Customer Profiles

In [704]:
profile_orig.head(3)

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN


There are disproportionately many customers with age 118 (12%), and for these the data seem incomplete. So we remove the respective data points.

In [705]:
fig = px.histogram(profile_orig, x='age', width=600, height=400, 
                   template = 'plotly_white', color_discrete_sequence=px.colors.qualitative.Dark24, labels={'age':'Age'})
fig.update_layout(yaxis_title="Number of customers")
fig.show()

In [706]:
profile_orig[profile_orig['age']==118]['age'].count() / profile_orig.shape[0]

0.12794117647058822

In [707]:
profile_orig[profile_orig['age']==118]

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN
7,None,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN
...,...,...,...,...,...
16980,None,118,5c686d09ca4d475a8f750f2ba07e0440,20160901,NaN
16982,None,118,d9ca82f550ac4ee58b6299cf1e5c824a,20160415,NaN
16989,None,118,ca45ee1883624304bac1e4c8a114f045,20180305,NaN
16991,None,118,a9a20fa8b5504360beb4e7c8712f8306,20160116,NaN


In [708]:
# Create dataframe with person IDs as index and readable pseudonym as column
def pseudonym_df(profile_df):
    '''
    Helper function that creates a dataframe with person IDs as index and clear pseudonyms as columns. 
    Pseudonyms are P1, P2, etc. 
    '''
    id_df = pd.DataFrame(profile_df['id'])
    id_df.set_index('id', inplace=True)
    id_df.insert(0, 'pseudonym', range(0, len(id_df)))
    id_df['pseudonym'] = 'P' + id_df['pseudonym'].astype(str)
    return id_df

id_df = pseudonym_df(profile_orig)


def clean_profile (profile_df):
    '''
    Cleans the profile data by assigning clearer person IDs, removing customers with age 118, 
    renaming 'id' to 'person_id', adding 'member_since' which contains just the relevant year from 'became_member_on',
    and by assigning appropriate ranges for age and income.
     
    Returns the cleaned profile data.
    '''
    profile = profile_df.copy()
    
    # Rename 'id' to 'person_id'
    profile.rename(columns={'id':'person_id'}, inplace=True)
    
    # Remove customers with age 118
    age_118 = profile[profile['age'] == 118]
    profile.drop(index=age_118.index, inplace=True) 
    
    # Assign each person the corresponding pseudonym
    profile['person_id'] = profile.apply(lambda x : id_df.loc[x.person_id], axis=1)
    
    # Replace 'became_member_on' with 'member_since'
    profile['member_since'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
    profile['member_since'] = pd.DatetimeIndex(profile['member_since']).year
    profile.drop(columns=['became_member_on'], inplace=True)
    
    #Assign age ranges
    labels = ['10-19','20-29', '30-39', '40-49', '50-59','60-69', '70-79', '80-89', '90-99', '100-109']
    profile['age_range'] = pd.cut(profile['age'], bins=range(10,120,10), right=False, labels=labels)
    
    # Assign income ranges 
    profile['income_range'] = pd.cut(profile['income'], bins=range(30000,140000,10000), right=False,\
                                    labels=['30-39K','40-49K', '50-59K', '60-69K', '70-79K','80-89K', '90-99K',\
                                            '100-109K', '110-119K', '>=120K'])
    
    return profile

profile = clean_profile(profile_orig)

In [709]:
profile.head()

,gender,age,person_id,income,member_since,age_range,income_range
1,F,55,P1,112000.0,2017,50-59,110-119K
3,F,75,P3,100000.0,2017,70-79,100-109K
5,M,68,P5,70000.0,2018,60-69,70-79K
8,M,65,P8,53000.0,2018,60-69,50-59K
12,M,58,P12,51000.0,2017,50-59,50-59K


In [710]:
# Calculate gender distribution
f_percentage = len(profile[profile['gender']=='F']) / len(profile)
m_percentage = len(profile[profile['gender']=='M']) / len(profile)
o_percentage = len(profile[profile['gender']=='O']) / len(profile)
print(f_percentage, m_percentage, o_percentage)

0.4134232715008432 0.5722765598650927 0.014300168634064081


In [711]:
# Plot gender distribution 
fig = px.histogram(profile, x='gender', width=300, height=300, 
                   template = 'plotly_white', color_discrete_sequence=px.colors.qualitative.Dark24, 
                  labels={'gender':'Gender', 'count':'Number of Customers'})
fig.show()

In [712]:
# Calculate age distribution in general

In [713]:
profile['age'].mean()

54.39352445193929

In [714]:
pd.DataFrame(profile.groupby(['gender']).mean()['age'].astype(int))

,age
gender,
F,57
M,52
O,54


In [715]:
# Plot age distribution in general
fig = px.histogram(profile, x='age', width=600, height=400, nbins=24,
                   template = 'plotly_white', color_discrete_sequence=px.colors.qualitative.Dark24, 
                  )
fig.show()

In [716]:
# Plot age distribution by gender
fig = px.histogram(profile, x='age', facet_col='gender', width=1000, height=400, nbins=24, 
                  template = 'plotly_white', color_discrete_sequence=px.colors.qualitative.Dark24, 
                   title='Age Distribution of Customers', labels={'age':'Age'})
fig.update_layout(yaxis_title="Number of customers")
fig.show()

In [717]:
# Income distribution in general
mean_income = profile['income'].mean()
mean_income 

65404.9915682968

In [718]:
# Income distribution by gender 
pd.DataFrame(profile.groupby(['gender']).mean()['income'].astype(int)) 

,income
gender,
F,71306
M,61194
O,63287


In [719]:
# Plot income distribution by gender 
fig = px.histogram(profile, x='income', facet_col='gender', width=1000, height=400, nbins=24, 
                  template = 'plotly_white', color_discrete_sequence=px.colors.qualitative.Dark24, 
                   title='Income Distribution of Customers', labels={'income':'Income'})
fig.update_layout(yaxis_title="Number of customers")
fig.show()

## Transcript of purchasing events

In [720]:
#combined_df[(combined_df['event']=='offer received') & (combined_df['outcome']==2)][['offer_id', 'outcome']].head()

In [721]:
def clean_transcript (transcript_df):
    '''
    Cleans the transcript file by renaming 'person' to 'person_id', 'offer id' to 'offer_id', expanding the 'value' column, 
    replacing NaNs  apporiately, and assigning a clearer person_id.
    
    Returns the cleaned transcript file.
    '''
    transcript = transcript_df.copy()
    
    # Rename 'person' to 'person_id'
    transcript.rename(columns={'person':'person_id'}, inplace=True)
    
    # Replace 'offer id' with 'offer_id'in 'value' column
    transcript['value'] = transcript['value'].astype(str).replace({'offer id': 'offer_id'}, regex=True)
    
    # 'Value' column contains dict as string, convert to actual dict
    transcript['value'] = transcript['value'].apply(literal_eval)
    
    # Split 'value' column into colums 'offer_id', 'reward' and 'amount'
    transcript = transcript.join(pd.DataFrame(transcript.pop('value').values.tolist()))
    
    # Replace NaNs with String 'none'
    transcript[['offer_id', 'amount', 'reward']] = transcript[['offer_id', 'amount', 'reward']].fillna('none')
    
    # Replace 'none' by 0
    transcript['amount'].replace('none', 0, inplace=True)
    transcript['reward'].replace('none', 0, inplace=True)
    
    # Replace 'offer_id' by the corresponding pseudonym
    transcript['offer_id'] = transcript['offer_id'].map(offer_mapping)
    
    # Replace 'person_id' by the corresponding pseudonym
    transcript['person_id'] = transcript.apply(lambda x : id_df.loc[x.person_id], axis=1)
    
    return transcript
    
    
transcript = clean_transcript(transcript_orig)

In [722]:
transcript.head()

,person_id,event,time,offer_id,amount,reward
0,P3,offer received,0,B3,0.0,0.0
1,P4,offer received,0,D1,0.0,0.0
2,P5,offer received,0,D4,0.0,0.0
3,P6,offer received,0,D3,0.0,0.0
4,P7,offer received,0,B2,0.0,0.0


In [723]:
# Time range of the transcript logs: 714 hours, 29.75 days
transcript['time'].max() / 24

29.75

In [724]:
# Offers per person
transcript[transcript['event']=='offer received'].groupby('person_id').count().describe()['event']

count    16994.000000
mean         4.488467
std          1.073047
min          1.000000
25%          4.000000
50%          5.000000
75%          5.000000
max          6.000000
Name: event, dtype: float64

In [725]:
# Transactions per person
transcript[transcript['event']=='transaction'].groupby('person_id').count().describe()['event']

count    16578.000000
mean         8.381771
std          5.009822
min          1.000000
25%          5.000000
50%          7.000000
75%         11.000000
max         36.000000
Name: event, dtype: float64

## Combining portfolio, profile and transcript data

In order to combine the available information, we merge transcript, portfolio and profile data.

In [726]:
# Merge transcript and portfolio data
combined_df = transcript.merge(portfolio, left_on='offer_id', right_on='offer_id', how='left')

In [727]:
# Merge combined transcript and portfolio data with customer profile data
combined_df = combined_df.merge(profile, left_on='person_id', right_on='person_id', how='inner')

In [728]:
combined_df.columns

Index(['person_id', 'event', 'time', 'offer_id', 'amount', 'reward_x',
       'reward_y', 'difficulty', 'duration', 'offer_type', 'web', 'email',
       'mobile', 'social', 'gender', 'age', 'income', 'member_since',
       'age_range', 'income_range'],
      dtype='object')

In [729]:
combined_df.drop(columns=['reward_y'], inplace=True)
combined_df.rename(columns={'reward_x':'reward'}, inplace=True)

In [730]:
combined_df.head(3)

,person_id,event,time,offer_id,amount,reward,difficulty,duration,offer_type,web,email,mobile,social,gender,age,income,member_since,age_range,income_range
0,P3,offer received,0,B3,0.00,0.0,5.0,168.0,bogo,1.0,1.0,1.0,0.0,F,75,100000.0,2017,70-79,100-109K
1,P3,offer viewed,6,B3,0.00,0.0,5.0,168.0,bogo,1.0,1.0,1.0,0.0,F,75,100000.0,2017,70-79,100-109K
2,P3,transaction,132,NaN,19.89,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,75,100000.0,2017,70-79,100-109K


### Number of transactions by age and gender

In [731]:
# Creating a dataframe containing number of transactions by age

In [732]:
def persons_age_range(age_range):
    return combined_df[combined_df['age_range']==age_range]['person_id'].nunique()

trans_by_age = pd.DataFrame(combined_df[combined_df['event']=='transaction'].groupby('age_range').count()['event'])
trans_by_age['customer_count'] = trans_by_age.apply(lambda x: persons_age_range(x.name), axis=1)
trans_by_age['transactions_per_person'] = trans_by_age['event'] / trans_by_age['customer_count']

In [733]:

fig = px.bar(trans_by_age, x=trans_by_age.index, y='transactions_per_person', width=600, height=400, 
             title='Transactions per person by age', 
            labels={'age_range': 'Age range',
                     'transactions_per_person': 'Transactions per person'}, template = 'plotly_white', color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(showlegend=True)
fig.show()

In [734]:
# Creating a dataframe containing number of transactions by gender

In [735]:
def persons_gender(gender):
    return combined_df[combined_df['gender']==gender]['person_id'].nunique()

trans_by_gender = pd.DataFrame(combined_df[combined_df['event']=='transaction'].groupby('gender').count()['event'])
trans_by_gender['customer_count'] = trans_by_gender.apply(lambda x: persons_gender(x.name), axis=1)
trans_by_gender['transactions_per_person'] = trans_by_gender['event'] / trans_by_gender['customer_count']

In [736]:
fig = px.bar(trans_by_gender, x=trans_by_gender.index, y='transactions_per_person', width=400, height=400, 
             title='Transactions per person by gender', 
            labels={'transactions_per_person': 'Transactions per person',
                     'gender': 'Gender'}, template = 'plotly_white', color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(showlegend=True)
fig.show()

### Calculating viewed status, transaction information and outcome of offers

In the following we derive more information about the offers and transactions and extend our data sets accordingly.

* **Viewed status**: For completed offers, whether it was viewed or not.
* **Transaction information**: For transactions adds the transaction value (sales price without any offers), value_intent (sales price without only the viewed offers), offers that were completed viewed and unviewed through this transaction, and informationals that were confirmed (i.e. followed by a transaction within validity period).
* **Offer outcome**: For each received offer its outcome is added. Possible offer outcomes are:
 * `State 1` (**viewed and completed**): For BOGO and discount offers, this is the case when there is an 'offer viewed' event followed by an 'offer completed' event within the validity period. For informational offers no 'offer completed' is generated explicitly, but can be derieved when there is an 'offer viewed' event, followed by a 'transaction' event during the validity period. 
 * `State 2` (**not viewed and completed**): For BOGO and discount offers, this applies when there is a 'offer completed' event, but no 'offer viewed' event. For informationals, this is the case when there is a transaction event following an 'offer received' event within the validity period.
 * `State 3` (**viewed and not completed**): The offer was viewed, but there is no transaction event while the offer is valid.
 * `State 4` (**not viewed and not completed**): The offer was not viewed, and there is no eligible transaction event while the offer is valid.


In [737]:
def add_viewed_status(idx, person_id, event, time, offer_id):
    '''
    Adds the viewed status to 'offer completed' events, i.e. whether this offer was viewed before completing it or not.
    '''
    if event=='offer completed':
        # check if there was an 'offer viewed' event for this person and offer before the 'offer completed' event
        res = combined_df.loc[(combined_df['person_id'] == person_id) & (combined_df['offer_id'] == offer_id) & 
                             (combined_df['event'] == 'offer viewed') & (combined_df['time'] <= time)]
        if len(res)>0:
            return 'completed viewed'
        else:
            return 'completed unviewed'
    return event
        

In [738]:
def add_transaction_information(person_id, event, time, amount):
    '''
    Adds value, value_intent, completed_viewed, completed_unviewed, confirmed_informationals to 'transaction' events
        
    '''    
    if event=='transaction':
        
        # find all records for this person, then find the completed offers, both unviewed and viewed
        person_records = combined_df[combined_df['person_id']==person_id]
        
        offers = person_records[(person_records['time']==time) & ((person_records['event']=='completed unviewed') | (person_records['event']=='completed viewed'))]
        
        # all offers with the same timestamp, i.e. offers that were completed by this transaction
        #offers = person_records[(person_records['time']==time) & 
        #                        ((person_records['event']=='completed unviewed')) | (person_records['event']=='completed viewed')]
        
        offers_unviewed = offers[offers['event']=='completed unviewed']
        offers_viewed = offers[offers['event']=='completed viewed']
        
        # Amount is the final discounted amount
        # Additionally we can calculate the value of a purchase, 
        # which is the amount the customer would have paid without any offers
        value = amount
        for index, row in offers.iterrows():
            if row['offer_type']=='bogo':
                value += amount
            elif row['offer_type']=='discount':
                value += row['reward']
                
        value_intent = amount
        for _, row in offers_viewed.iterrows():
            if row['offer_type']=='bogo':
                value_intent += amount
            elif row['offer_type']=='discount':
                value_intent += row['reward']
        
        # get all offer ids, to link these in the DataFrame
        offer_ids = list(offers_unviewed['offer_id']) + list(offers_viewed['offer_id'])
        
        # link completed offers
        completed_viewed = list(offers_viewed['offer_id'])
        completed_unviewed = list(offers_unviewed['offer_id'])
        
        # link informational offers followed by a transaction within validity duration
        confirmed_informationals = list(person_records[(person_records['event']=='offer viewed') & 
                                                       (person_records['offer_type']=='informational') & 
                                                       (time >= person_records['time']) &
                                                       (time <= person_records['time'] + person_records['duration'])]['offer_id'])
                  
        return value, value_intent, completed_viewed, completed_unviewed, confirmed_informationals
        
    return amount, amount, '[]', '[]', '[]'


In [739]:
# 1: 'Confirmed viewed' or 'confirmed informational'
# 2: 'Confirmed unviewed'
# 3:viewed and not completed
# Else: 4 -> not viewed and not accepted
def add_offer_outcome(idx, person_id, event, time, offer_type, offer_id, duration):
    '''
    Adds the outcome state 'offer received' events
    '''
    if event=='offer received':
        # Find all records for this person in the valid time range
        person_records = combined_df[(combined_df['person_id']==person_id) & (combined_df['time'] >= time) \
                                    & (combined_df['time'] <= time + duration)]
                  
        # If there are 'completed viewed' or 'confirmed informational' records -> outcome state 1
        mask = person_records['completed viewed'].apply(lambda x: offer_id in x) \
        | person_records['confirmed informationals'].apply(lambda x: offer_id in x)
        outcome_1_records = person_records[mask]
        if len(outcome_1_records)>0: 
            return 1 
        
        # 'Completed unviewed' -> outcome state 2
        mask = person_records['completed unviewed'].apply(lambda x: offer_id in x) \
            & person_records['completed unviewed'].apply(lambda x: offer_id in x) 
        
        outcome_2_records = person_records[mask]
        
        if len(outcome_2_records)>0:
            return 2
        
        # For informationals, find transaction events of this person within the validity duration
        # If there are any, the outcome of this informational is 2 ('completed unviewed')
        if offer_type=='informational':
            completed_informationals = combined_df[(combined_df['person_id']==person_id) \
                                               & (combined_df['event']=='transaction') \
                                               & (combined_df['time']<=time+duration)\
                                              & (combined_df['time']>=time)]                                  
            if len(completed_informationals)>0:
                return 2
        
        # Offer was viewed within validity duration, but not completed -> outcome state 3
        outcome_3_records = person_records[(person_records['offer_id']==offer_id) & (person_records['event']=='offer viewed')]
        if len(outcome_3_records)>0:
            return 3
        
        # Offer was not viewed and not completed
        else:
            return 4
        

In [741]:
# Either call the above functions, or load pre-calculated data
recalculate = False # Set to True if calculation should be done from scratch, but calculation takes about one hour
if recalculate:
    combined_df['event'] = combined_df.apply(
        lambda x : add_viewed_status(x.name, x.person_id, x.event, x.time, x.offer_id), axis=1)
    
    combined_df[['value', 'value_intent', 'completed viewed', 'completed unviewed', 'confirmed informationals']] = combined_df.apply(
        lambda x : add_transaction_information(x.person_id, x.event, x.time, x.amount), axis=1, result_type='expand')
    
    combined_df['outcome']= combined_df.apply(
        lambda x : add_offer_outcome(x.name, x.person_id, x.event, x.time, x.offer_type, x.offer_id, x.duration), axis=1)
     
    combined_df.to_csv('data/combined_df.csv')

else:
    # load combined_df from file
    combined_zip = zipfile.ZipFile('data/combined_df.zip')
    combined_df = pd.read_csv(combined_zip.open('combined_df.csv'), index_col='Unnamed: 0')

In [742]:
combined_df.head()

,person_id,event,time,offer_id,amount,reward,difficulty,duration,offer_type,web,...,income,member_since,age_range,income_range,value,value_intent,completed viewed,completed unviewed,confirmed informationals,outcome
0,P3,offer received,0,B3,0.00,0.0,5.0,168.0,bogo,1.0,...,100000.0,2017,70-79,100-109K,0.00,0.00,[],[],[],1.0
1,P3,offer viewed,6,B3,0.00,0.0,5.0,168.0,bogo,1.0,...,100000.0,2017,70-79,100-109K,0.00,0.00,[],[],[],NaN
2,P3,transaction,132,NaN,19.89,0.0,NaN,NaN,NaN,NaN,...,100000.0,2017,70-79,100-109K,39.78,39.78,['B3'],[],[],NaN
3,P3,completed viewed,132,B3,0.00,5.0,5.0,168.0,bogo,1.0,...,100000.0,2017,70-79,100-109K,0.00,0.00,[],[],[],NaN
4,P3,transaction,144,NaN,17.78,0.0,NaN,NaN,NaN,NaN,...,100000.0,2017,70-79,100-109K,17.78,17.78,[],[],[],NaN


### Exploring offer outcome

In [743]:
# Build dataframes counting transactions by outcome
outcome_df = pd.DataFrame(combined_df.groupby(['outcome']).count()['person_id']).rename(columns={'person_id':'count'})
outcome_df['percentage'] = round(outcome_df['count'] / len(combined_df[combined_df['event']=='offer received'])*100, 2)
outcome_df.index = outcome_df.index.astype(int)
outcome_df

,count,percentage
outcome,,
1,29398,44.21
2,11487,17.27
3,16351,24.59
4,9265,13.93


In [744]:
outcome_df.loc[1, 'outcome state'] = 'viewed + completed'
outcome_df.loc[2, 'outcome state'] = 'not viewed + completed'
outcome_df.loc[3, 'outcome state'] = 'viewed + not completed'
outcome_df.loc[4, 'outcome state'] = 'not viewed + not completed'
outcome_df

,count,percentage,outcome state
outcome,,,
1,29398,44.21,viewed + completed
2,11487,17.27,not viewed + completed
3,16351,24.59,viewed + not completed
4,9265,13.93,not viewed + not completed


In [745]:
# Plot percentage for each outcome state
fig = px.bar(outcome_df, x=outcome_df.index, y='percentage', width=400, height=400, title='Outcome of received offers', 
            labels={'outcome': 'Offer outcome',
                     'percentage': 'Percentage of offers'}, template = 'plotly_white', color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(showlegend=True)
fig.show()

In [746]:
# Build dataframe for outcome by gender
outcome_df_gender = pd.DataFrame(combined_df.groupby(['outcome', 'gender']).count()['person_id']).rename(columns={'person_id':'offer_count'})
outcome_df_gender = outcome_df_gender.reset_index()
outcome_df_gender.head()

,outcome,gender,offer_count
0,1.0,F,13404
1,1.0,M,15497
2,1.0,O,497
3,2.0,F,5514
4,2.0,M,5848


In [747]:
# len(combined_df[(combined_df['event']=='offer received') & (combined_df['gender']==gender)])

def outcome_gender_percentage(outcome, gender, offer_count):
    '''
    Helper function to return the percentage of all customers for this gender and outcome
    '''
    overall = len(combined_df[(combined_df['event']=='offer received') & (combined_df['gender']==gender)])
    return offer_count / overall

outcome_df_gender['percentage'] = outcome_df_gender.apply(lambda x : outcome_gender_percentage(x.outcome, x.gender, x.offer_count), axis=1)

In [748]:
outcome_df_gender.head()

,outcome,gender,offer_count,percentage
0,1.0,F,13404,0.488199
1,1.0,M,15497,0.406436
2,1.0,O,497,0.542576
3,2.0,F,5514,0.200830
4,2.0,M,5848,0.153374


In [749]:
# Plot offer outcome by gneder
fig = px.bar(outcome_df_gender, x='outcome', y='percentage', width=400, height=400, title='Offer outcome by gender',
             color='gender', barmode='group',  template = 'plotly_white',  
             color_discrete_sequence=px.colors.qualitative.Dark24, 
            labels={'outcome':'Outcome', 'percentage':'Percentage of offers'})
fig.show()

Based on the merged and extended data, we can calculate various metrics for offers:
* **View Rate**: 'viewed and not completed' + 'viewed and completed' / received -> outcome1 + outcome3 / received
* **Completion Rate**: 'viewed and completed' + 'not viewed and completed' / received -> outcome1 + outcome2 / received
* **Take Rate**: 'viewed and completed' / 'viewed' -> outcome1 / viewed
* **Reject Rate**: 'viewed and uncompleted' / 'viewed' -> outcome3 / viewed

In [750]:
# Complete various metrics for the 
result_df = pd.DataFrame(data=portfolio[['offer_id', 'offer_type']])


def view_rate(offer_id):
    '''
    Returns the View Rate for the given offer_id
    '''
    outcome_1_3 = len(combined_df[(combined_df['offer_id']==offer_id) \
                                  & (combined_df['event']=='offer received')
                                  & ((combined_df['outcome']==1) | (combined_df['outcome']==3))])
    received = len(combined_df[(combined_df['offer_id']==offer_id) & (combined_df['event']=='offer received')])
    return outcome_1_3 / received

result_df['view rate'] = round(result_df.apply(lambda x : view_rate(x.offer_id), axis=1)*100, 2)

def completion_rate(offer_id):
    '''
    Returns the Completion Rate for the given offer_id
    '''
    
    outcome_1_2 = len(combined_df[(combined_df['offer_id']==offer_id) \
                                  & (combined_df['event']=='offer received')
                                  & ((combined_df['outcome']==1) | (combined_df['outcome']==2))])
    received = len(combined_df[(combined_df['offer_id']==offer_id) & (combined_df['event']=='offer received')])
    return outcome_1_2 / received

result_df['completion rate'] = round(result_df.apply(lambda x : completion_rate(x.offer_id), axis=1)*100, 2)

def take_rate(offer_id):
    '''
    Returns the Take Rate for the given offer_id
    '''
    
    outcome_1 = len(combined_df[(combined_df['offer_id']==offer_id) \
                                  & (combined_df['event']=='offer received')
                                  & (combined_df['outcome']==1)])
    viewed = len(combined_df[(combined_df['offer_id']==offer_id) \
                             & (combined_df['event']=='offer viewed')])
    return outcome_1 / viewed

result_df['take rate'] = round(result_df.apply(lambda x : take_rate(x.offer_id), axis=1)*100, 2)
result_df = result_df.sort_values(by='take rate', ascending=False)

def reject_rate(offer_id):
    '''
    Returns the Reject Rate for the given offer_id
    '''
    outcome_3 = len(combined_df[(combined_df['offer_id']==offer_id) \
                                  & (combined_df['event']=='offer received')
                                  & (combined_df['outcome']==3)])
    viewed = len(combined_df[(combined_df['offer_id']==offer_id) \
                             & (combined_df['event']=='offer viewed')])
    return outcome_3 / viewed

# Build dataframe listing the above metrics for each offer
result_df['reject rate'] = round(result_df.apply(lambda x : reject_rate(x.offer_id), axis=1)*100, 2)
result_df


,offer_id,offer_type,view rate,completion rate,take rate,reject rate
6,D3,discount,91.55,75.35,70.95,24.10
5,D2,discount,89.57,73.49,67.60,25.85
4,D1,discount,32.16,50.58,67.27,30.38
9,D4,discount,47.25,59.04,65.84,24.71
3,B3,bogo,47.28,62.65,65.45,24.89
2,I1,informational,44.63,62.84,57.07,28.13
7,I2,informational,75.52,63.59,56.29,29.13
8,B4,bogo,88.75,62.39,55.82,36.67
0,B1,bogo,81.86,54.72,47.35,45.37
1,B2,bogo,90.19,50.20,45.35,48.60


In [751]:
# Plot take rate by offer id
fig = px.bar(result_df, x='offer_id', y='take rate', width=600, height=400, title='Take rate for offers', 
             template = 'plotly_white', labels={'take rate':'Take rate (%'')', 'offer_id':''}, 
            color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(showlegend=True)
fig.show()

In [752]:
# Take rate by gender and offer_id

a1 = np.array(['B1', 'B1', 'B1', 'B2', 'B2', 'B2', 'B3', 'B3', 'B3', 'B4', 'B4', 'B4', 'D1', 'D1', 'D1', 'D2', 'D2', 'D2', 'D3', 'D3','D3', 'D4', 'D4', 'D4', 'I1', 'I1', 'I1', 'I2', 'I2', 'I2'])
a2 = np.array(['M', 'F', 'O', 'M', 'F', 'O', 'M', 'F', 'O', 'M', 'F', 'O', 'M', 'F', 'O', 'M', 'F', 'O', 'M', 'F', 'O', 'M', 'F', 'O', 'M', 'F', 'O', 'M', 'F', 'O'])
arrays=[a1, a2]

take_rate_gender_id_df = pd.DataFrame(index=arrays)
take_rate_gender_id_df['take rate'] = 0

def take_rate_gender_id(offer_id, gender):
    outcome_1 = len(combined_df[(combined_df['offer_id']==offer_id) \
                                  & (combined_df['event']=='offer received')
                                  & (combined_df['outcome']==1) \
                                  & (combined_df['gender']==gender)])
    viewed = len(combined_df[(combined_df['offer_id']==offer_id) \
                             & (combined_df['event']=='offer viewed') \
                             & (combined_df['gender']==gender)])
    return outcome_1 / viewed

take_rate_gender_id_df['take rate'] = round(take_rate_gender_id_df.apply(lambda x : take_rate_gender_id(x.name[0], x.name[1]), axis=1)*100, 2)
take_rate_gender_id_df = take_rate_gender_id_df.reset_index().rename(columns={'level_0':'offer_id', 'level_1':'gender'})

In [753]:
# Take rate by gender
take_rate_gender_df = round(take_rate_gender_id_df.groupby('gender').mean(), 2)
take_rate_gender_df

,take rate
gender,
F,64.72
M,56.27
O,64.47


In [754]:
# Plot take rate by gender and offer_id
fig = px.bar(take_rate_gender_id_df, x='offer_id', y='take rate', width=800, height=400, title='Take rate by gender and offer',
             color='gender', barmode='group',  template = 'plotly_white',  
             color_discrete_sequence=px.colors.qualitative.Dark24, 
            labels={'offer_id':'Offer', 'take rate':'Take rate (%)'})
fig.show()

In [755]:
# Take rate by income and offer id

In [756]:
take_rate_income_df

,income range,take rate
0,30-39K,0.337246
1,40-49K,0.344684
2,50-59K,0.451532
3,60-69K,0.465547
4,70-79K,0.476354
5,80-89K,0.527074
6,90-99K,0.517114
7,100-109K,0.451487
8,110-119K,0.457354
9,>=120K,0.421875


In [757]:
# Build dataframe showing take rate by income range

def take_rate_income(income_range):
    '''
    Helper function returning the take rate for the given income range
    '''
    offers_df = combined_df[(combined_df['event']=='offer received') & (combined_df['income_range']==income_range)]
    if len(offers_df)>0:
        return len(offers_df[offers_df['outcome']==1]) / len(offers_df)
    else:
        return 0;

ranges = ['30-39K', '40-49K', '50-59K', '60-69K', '70-79K', '80-89K', '90-99K', '100-109K', '110-119K', '>=120K'] 
rate = []
for r in ranges:
    rate.append(take_rate_income(r))
take_rate_income_df = pd.DataFrame({'income range': ranges, 'take rate': rate})

In [758]:
# Plot take rate by gender and offer_id
fig = px.bar(take_rate_income_df, x='income range', y='take rate', width=600, height=400, title='Take rate by income',
             barmode='group',  template = 'plotly_white',  
             color_discrete_sequence=px.colors.qualitative.Dark24, 
            labels={'income range':'Income range', 'take rate':'Take rate (%)'})
fig.show()

### Exploring purchasing behavior

In [759]:
# Define dataframe for transactions only, having the specified columns
cols = ['person_id', 'event', 'time', 'age_range', 'gender', 'income_range', 'member_since', 'amount', 'value', 'completed viewed', 
        'completed unviewed', 'confirmed informationals']
transactions_df = combined_df[combined_df['event']=='transaction'][cols]
transactions_df.head()

,person_id,event,time,age_range,gender,income_range,member_since,amount,value,completed viewed,completed unviewed,confirmed informationals
2,P3,transaction,132,70-79,F,100-109K,2017,19.89,39.78,['B3'],[],[]
4,P3,transaction,144,70-79,F,100-109K,2017,17.78,17.78,[],[],[]
7,P3,transaction,222,70-79,F,100-109K,2017,19.67,19.67,[],[],['I2']
8,P3,transaction,240,70-79,F,100-109K,2017,29.72,29.72,[],[],['I2']
9,P3,transaction,378,70-79,F,100-109K,2017,23.93,23.93,[],[],[]


In [760]:
# Average transaction amount
transactions_df['amount'].mean()

13.996324531894219

In [761]:
# Average transaction amount by gender
pd.DataFrame(transactions_df.groupby(['gender']).mean()['amount'])

,amount
gender,
F,17.490077
M,11.606600
O,14.798731


In [762]:
# Plot transaction amount by income 
amount_by_income_df = pd.DataFrame(transactions_df.groupby('income_range')['amount'].mean())
amount_by_income_df = amount_by_income_df.reindex(labels=ranges)

fig = px.bar(amount_by_income_df, x=amount_by_income_df.index, y='amount', width=600, height=400, 
             title='Transaction amount by income', labels={'income_range':'Income range', 'amount':'Transaction amount'}, 
            color_discrete_sequence=px.colors.qualitative.Dark24, template = 'plotly_white')
fig.show()

In [763]:
# Plot transaction amount by age 
amount_by_age_df = pd.DataFrame(transactions_df.groupby('age_range')['amount'].mean())
age_labels = ['10-19','20-29', '30-39', '40-49', '50-59','60-69', '70-79', '80-89', '90-99', '100-109']
amount_by_age_df = amount_by_age_df.reindex(labels=age_labels)
    

fig = px.bar(amount_by_age_df, x=amount_by_age_df.index, y='amount', width=600, height=400, 
             title='Transaction amount by age', labels={'age_range':'Age', 'amount':'Transaction amount'}, 
            color_discrete_sequence=px.colors.qualitative.Dark24, template = 'plotly_white')
fig.show()

In [764]:
# Compare transactions with and without offers

transactions_df['completed viewed'] = transactions_df['completed viewed'].astype(str)
transactions_df['completed unviewed'] = transactions_df['completed unviewed'].astype(str)
transactions_df['confirmed informationals'] = transactions_df['confirmed informationals'].astype(str)

transactions_without = transactions_df[(transactions_df['completed viewed']=='[]') & (transactions_df['completed unviewed']=='[]') & (transactions_df['confirmed informationals']=='[]')]
transactions_with = transactions_df[(transactions_df['completed viewed']!='[]') | (transactions_df['completed unviewed']!='[]') | (transactions_df['confirmed informationals']!='[]')]

In [765]:
# Average transaction amount with and without offers
print(transactions_without['amount'].mean(), transactions_with['amount'].mean())

11.754955262188561 18.735694221105653


In [766]:
# Average transaction value for transactions with offers, 
# i.e. the (assumed) amount the customer would have had to pay without any offers
transactions_with['value'].mean()

28.112761055276444

In [767]:
# Difference between final discounted amount, and the (assumed) transaction value
# This difference is the loss Starbucks makes by granting the discounts 
transactions_with['value'].mean() - transactions_with['amount'].mean()

9.37706683417079

In [768]:
# Overview on transaction amount with and without offer
trans = ['with offer', 'without offer']
amount = [transactions_with['amount'].mean(), transactions_without['amount'].mean()]
transactions_df = pd.DataFrame({'transactions':trans, 'amount':amount})
transactions_df

,transactions,amount
0,with offer,18.735694
1,without offer,11.754955


In [769]:
# Build DataFrame for counting received offers and transactions by person
transactions_offers_count_df = pd.DataFrame(combined_df['person_id'].unique(), columns=['person_id'])

In [770]:
# Add number of offers per person to combined_df
offers_per_person_df = pd.DataFrame(combined_df[combined_df['event']=='offer received'].groupby(
    'person_id').count()['offer_id'])
offers_per_person_df.rename(columns={'offer_id':'offer_count'}, inplace=True)
transactions_offers_count_df = transactions_offers_count_df.merge(offers_per_person_df, left_on='person_id', right_on='person_id', how='left')

In [771]:
# Add number of transactions per person to combined_df
transactions_per_person_df = pd.DataFrame(combined_df[combined_df['event']=='transaction'].groupby(
    'person_id').count()['event'])
transactions_per_person_df.rename(columns={'event':'transaction_count'}, inplace=True)
transactions_offers_count_df = transactions_offers_count_df.merge(transactions_per_person_df, left_on='person_id', right_on='person_id', how='left')

In [772]:
# Plot number of transactions by number of offers 
transactions_by_offers = transactions_offers_count_df.groupby('offer_count').mean()

fig = px.bar(transactions_by_offers, x=transactions_by_offers.index, y='transaction_count', width=600, height=400, 
             title='Number of transactions by number of offers', labels={'offer_count':'Number of offers', 
                                                                         'transaction_count':'Number of transactions'}, 
            color_discrete_sequence=px.colors.qualitative.Dark24, template = 'plotly_white')
fig.show()

## Predicting offer take rate

### Preparing the data

In [773]:
# One-hot encode gender
gender_dummies = pd.get_dummies(combined_df['gender'])

# One-hot encode member_since
year_dummies = pd.get_dummies(combined_df['member_since'])

# One-hot encode age_range
age_dummies = pd.get_dummies(combined_df['age_range'])

# One-hot encode income_range
income_dummies = pd.get_dummies(combined_df['income_range'])

# One-hot encode offer_type
offer_dummies = pd.get_dummies(combined_df['offer_type'])

# Extend combined_df
combined_df = pd.concat([combined_df, gender_dummies, year_dummies, age_dummies, income_dummies, offer_dummies], axis=1)



In [774]:
# Rename columns with int names
col_names = {
    2013 : 'member_2013', 
    2014 : 'member_2014', 
    2015 : 'member_2015',
    2016 : 'member_2016',
    2017 : 'member_2017', 
    2018 : 'member_2018'
}
combined_df.rename(columns=col_names, inplace=True)

In [775]:
combined_df.columns

Index(['person_id', 'event', 'time', 'offer_id', 'amount', 'reward',
       'difficulty', 'duration', 'offer_type', 'web', 'email', 'mobile',
       'social', 'gender', 'age', 'income', 'member_since', 'age_range',
       'income_range', 'value', 'value_intent', 'completed viewed',
       'completed unviewed', 'confirmed informationals', 'outcome', 'F', 'M',
       'O', 'member_2013', 'member_2014', 'member_2015', 'member_2016',
       'member_2017', 'member_2018', '10-19', '100-109', '20-29', '30-39',
       '40-49', '50-59', '60-69', '70-79', '80-89', '90-99', '100-109K',
       '110-119K', '30-39K', '40-49K', '50-59K', '60-69K', '70-79K', '80-89K',
       '90-99K', '>=120K', 'bogo', 'discount', 'informational'],
      dtype='object')

In [776]:
# Drop columns not required for learning the model
drop_cols = ['person_id', 'event', 'amount', 'time', 'offer_id', 'offer_type', 'gender', 'age', 'income', 'age_range', 'income_range', 
            'value', 'value_intent', 'completed viewed', 'completed unviewed', 'confirmed informationals']
offers = combined_df[combined_df['event']=='offer received'].copy()
offers.drop(columns=drop_cols, inplace=True)

# Predict whether outcome is 1 (viewed + completed) or not
offers['outcome_1'] = (offers['outcome'] == 1).astype(int)

# Delete output columns from feature list, build target vector from output_1 column
X = offers.drop(columns=['outcome', 'outcome_1'])
y = offers['outcome_1']

### RandomForestClassifier with default parameters

In [777]:
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [778]:
# Use RandomForestClassifier with default params
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [779]:
# Predict on the test set
y_pred = classifier.predict(X_test)

In [780]:
# Classification_report(y_test, y_pred, output_dict = True)
default_report = classification_report(y_test, y_pred, output_dict = True) 
display(pd.DataFrame(default_report))

,0,1,accuracy,macro avg,weighted avg
precision,0.685088,0.631555,0.663599,0.658322,0.661426
recall,0.734936,0.573547,0.663599,0.654242,0.663599
f1-score,0.709137,0.601155,0.663599,0.655146,0.661407
support,9277.000000,7349.000000,0.663599,16626.000000,16626.000000


In [781]:
# Create DataFrame for comparison of different parameter sets and classifiers
df_comparison = (pd.DataFrame(default_report))['weighted avg']
df_comparison = pd.DataFrame(df_comparison).T.rename({'weighted avg':'RandomForestClassifier (default parameters)'})
df_comparison

,precision,recall,f1-score,support
RandomForestClassifier (default parameters),0.661426,0.663599,0.661407,16626.0


### Parameter tuning with GridSearchCV 

In [669]:
from sklearn.model_selection import GridSearchCV
param_grid = {
        "max_depth": [None, 3, 10],
        "min_samples_split" : [2, 5],
        "max_features" : [None, 'log2', 'sqrt'],  
        "min_samples_leaf" : [1, 2, 10] 
    }

cv = GridSearchCV(classifier, param_grid, refit='f1_weighted', return_train_score=True, n_jobs=-1, cv =3)
cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 3, 10],
                         'max_features': [None, 'log2', 'sqrt'],
                         'min_samples_leaf': [1, 2, 10],
                         'min_samples_split': [2, 5]},
             refit='f1_weighted', return_train_score=True)

In [670]:
cv_report = classification_report(y_test,cv.best_estimator_.predict(X_test), output_dict = True)
display(pd.DataFrame(cv_report))

,0,1,accuracy,macro avg,weighted avg
precision,0.696630,0.665656,0.684951,0.681143,0.683003
recall,0.774890,0.570472,0.684951,0.672681,0.684951
f1-score,0.733679,0.614399,0.684951,0.674039,0.681199
support,9311.000000,7315.000000,0.684951,16626.000000,16626.000000


In [671]:
# Results of the individual configurations
pd.set_option('display.max_rows', 100)
cv_results = pd.DataFrame.from_dict(cv.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,44.507291,2.913597,1.401071,0.097002,None,None,1,2,"{'max_depth': None, 'max_features': None, 'min...",0.661113,0.655398,0.667729,0.661414,0.005039,46,0.747398,0.749444,0.742767,0.746536,0.002793
1,43.854991,1.124694,1.320069,0.008149,None,None,1,5,"{'max_depth': None, 'max_features': None, 'min...",0.660391,0.656722,0.668511,0.661875,0.004926,45,0.746617,0.748541,0.741955,0.745704,0.002765
2,42.588478,1.245111,1.277549,0.064554,None,None,2,2,"{'max_depth': None, 'max_features': None, 'min...",0.665865,0.657684,0.672301,0.665283,0.005981,33,0.742105,0.743940,0.737744,0.741263,0.002598
3,42.082428,0.213621,1.103904,0.002071,None,None,2,5,"{'max_depth': None, 'max_features': None, 'min...",0.665323,0.658346,0.671398,0.665023,0.005333,35,0.741955,0.743579,0.736962,0.740832,0.002815
4,37.960349,0.164192,0.967602,0.035700,None,None,10,2,"{'max_depth': None, 'max_features': None, 'min...",0.681564,0.673925,0.686436,0.680642,0.005149,21,0.717955,0.720872,0.713444,0.717424,0.003056
5,38.391211,0.458936,1.184685,0.208594,None,None,10,5,"{'max_depth': None, 'max_features': None, 'min...",0.680902,0.673684,0.685835,0.680140,0.004990,23,0.717774,0.720000,0.713293,0.717023,0.002789
6,14.869081,0.827399,1.602837,0.093265,None,log2,1,2,"{'max_depth': None, 'max_features': 'log2', 'm...",0.659850,0.655038,0.668571,0.661153,0.005601,47,0.747398,0.749444,0.742767,0.746536,0.002793
7,14.235944,0.449401,1.491394,0.225596,None,log2,1,5,"{'max_depth': None, 'max_features': 'log2', 'm...",0.663158,0.657083,0.673323,0.664521,0.006700,40,0.745564,0.748120,0.741293,0.744992,0.002816
8,12.478480,0.430477,1.220046,0.100257,None,log2,2,2,"{'max_depth': None, 'max_features': 'log2', 'm...",0.673143,0.666947,0.682105,0.674065,0.006222,26,0.736541,0.737835,0.732301,0.735559,0.002364
9,12.730870,0.444352,1.015657,0.036060,None,log2,2,5,"{'max_depth': None, 'max_features': 'log2', 'm...",0.673805,0.667188,0.681624,0.674206,0.005900,25,0.735940,0.737143,0.731398,0.734827,0.002474


In [674]:
# Add to comparison DataFrame
row_cv = pd.DataFrame((pd.DataFrame(cv_report))['weighted avg']).T
df_comparison = pd.concat([df_comparison, row_cv]).rename({'weighted avg':'RandomForestClassifier (GridSearchCV)'})
df_comparison.drop_duplicates()

,precision,recall,f1-score,support
RandomForestClassifier (default parameters),0.664214,0.666246,0.664448,16626.0
RandomForestClassifier (GridSearchCV),0.683003,0.684951,0.681199,16626.0


### Adding personal take rate as additional feature

Next, we are adding the personal take rate (PTR) for each person, however, this does not improve the classification results.

In [783]:
def personal_take_rate(person_id):
    '''
    Helper function that returns the personal take rate (PTR) for each person
    '''
    received_offers = combined_df[(combined_df['person_id']==person_id) & (combined_df['event']=='offer received')]
    completed_offers = received_offers[received_offers['outcome']==1]
    if len(received_offers) > 0:
        return len(completed_offers) / len(received_offers)
    else:
        return 0

# Recalculate or load the PTR information
recalculate = False # Set to True if take rates should be calculated from scratch; takes about two hours 
if recalculate:
    take_rates = []
    persons = combined_df['person_id'].values
    for person in persons:
        take_rates.append(personal_take_rate(person))
else:
    person_take_rates_zip = zipfile.ZipFile('data/person_take_rates_df.zip')
    take_rates = pd.read_csv(person_take_rates_zip.open('person_take_rates_df.csv'), index_col='Unnamed: 0')


In [784]:
take_rates = take_rates.groupby('person_id').max()

In [785]:
# Merge combined df with person_rate_df
combined_df_PTR = combined_df.merge(take_rates, left_on='person_id', right_on='person_id', how='left')
combined_df_PTR.head(5)

,person_id,event,time,offer_id,amount,reward,difficulty,duration,offer_type,web,...,50-59K,60-69K,70-79K,80-89K,90-99K,>=120K,bogo,discount,informational,person_take_rate
0,P3,offer received,0,B3,0.00,0.0,5.0,168.0,bogo,1.0,...,0,0,0,0,0,0,1,0,0,0.75
1,P3,offer viewed,6,B3,0.00,0.0,5.0,168.0,bogo,1.0,...,0,0,0,0,0,0,1,0,0,0.75
2,P3,transaction,132,NaN,19.89,0.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.75
3,P3,completed viewed,132,B3,0.00,5.0,5.0,168.0,bogo,1.0,...,0,0,0,0,0,0,1,0,0,0.75
4,P3,transaction,144,NaN,17.78,0.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.75


In [678]:
# Drop columns not required for learning the model
drop_cols = ['person_id', 'event', 'amount', 'time', 'offer_id', 'offer_type', 'gender', 'age', 'income', 'age_range', 'income_range', 
            'value', 'value_intent', 'completed viewed', 'completed unviewed', 'confirmed informationals']
offers = combined_df_PTR[combined_df_PTR['event']=='offer received'].copy()
offers.drop(columns=drop_cols, inplace=True)

# Predict whether outcome is 1 (viewed + completed) or not
offers['outcome_1'] = (offers['outcome'] == 1).astype(int)

# Delete output columns from feature list, build target vector from output_1 column
X = offers.drop(columns=['outcome', 'outcome_1'])
y = offers['outcome_1']

In [679]:
offers

,reward,difficulty,duration,web,email,mobile,social,member_since,outcome,F,...,60-69K,70-79K,80-89K,90-99K,>=120K,bogo,discount,informational,person_take_rate,outcome_1
0,0.0,5.0,168.0,1.0,1.0,1.0,0.0,2017,1.0,1,...,0,0,0,0,0,1,0,0,0.75,1
5,0.0,0.0,72.0,0.0,1.0,1.0,1.0,2017,1.0,1,...,0,0,0,0,0,0,0,1,0.75,1
10,0.0,10.0,168.0,0.0,1.0,1.0,1.0,2017,1.0,1,...,0,0,0,0,0,1,0,0,0.75,1
12,0.0,5.0,120.0,1.0,1.0,1.0,1.0,2017,2.0,1,...,0,0,0,0,0,1,0,0,0.75,0
18,0.0,10.0,168.0,1.0,1.0,1.0,0.0,2018,3.0,0,...,0,1,0,0,0,0,1,0,0.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272752,0.0,10.0,120.0,1.0,1.0,1.0,1.0,2018,3.0,0,...,0,0,0,0,0,1,0,0,0.00,0
272754,0.0,10.0,168.0,1.0,1.0,1.0,0.0,2018,4.0,0,...,0,0,0,0,0,0,1,0,0.00,0
272756,0.0,0.0,96.0,1.0,1.0,1.0,0.0,2017,3.0,0,...,0,0,0,1,0,0,0,1,0.00,0
272758,0.0,10.0,120.0,1.0,1.0,1.0,1.0,2017,3.0,0,...,0,0,0,1,0,1,0,0,0.00,0


In [680]:
# Use RandomForestClassifier with default params
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [681]:
# Predict on the test set
y_pred = classifier.predict(X_test)

In [682]:
# Classification_report(y_test, y_pred, output_dict = True)
ptr_report = classification_report(y_test, y_pred, output_dict = True) 
display(pd.DataFrame(ptr_report))

,0,1,accuracy,macro avg,weighted avg
precision,0.689036,0.633168,0.666667,0.661102,0.664456
recall,0.737730,0.576213,0.666667,0.656971,0.666667
f1-score,0.712552,0.603350,0.666667,0.657951,0.664506
support,9311.000000,7315.000000,0.666667,16626.000000,16626.000000


In [683]:
# Add to comparison DataFrame
row_ptr = pd.DataFrame((pd.DataFrame(ptr_report))['weighted avg']).T
df_comparison = pd.concat([df_comparison, row_ptr]).rename({'weighted avg':'RandomForestClassifier (with PTR)'})
df_comparison[['precision', 'recall', 'f1-score']].drop_duplicates()

,precision,recall,f1-score
RandomForestClassifier (default parameters),0.664214,0.666246,0.664448
RandomForestClassifier (GridSearchCV),0.683003,0.684951,0.681199
RandomForestClassifier (with PTR),0.664456,0.666667,0.664506
